In [7]:
from rmgpy.kinetics.uncertainties import RateUncertainty
import rmgpy
import numpy as np
from rmgpy.molecule.molecule import *
from rmgpy.species import *
from rmgpy.chemkin import *
from rmgpy.data.rmg import RMGDatabase
from IPython.display import display
from rmgpy.data.thermo import ThermoLibrary
from rmgpy.rmg.react import react
from rmgpy.species import Species
from rmgpy.reaction import Reaction
from rmgpy.data.rmg import get_db
from rmgpy.molecule.group import Group
from rmgpy.kinetics.arrhenius import ArrheniusBM, Arrhenius
from rmgpy import settings
import time
import matplotlib.pyplot as plt
import matplotlib


In [6]:
import sys
sys.path.append('/Users/nora/Code/RMG-Py')

In [8]:
thermo_libs = [
'C1_C2_Fluorine', #adding Siddha's as first most trusted because this is the thermo library that Franklin used
'NCSU_C2_C8_PFAS', #adding Westmoreland's thermo as the second most trusted
'primaryThermoLibrary',
'Fluorine',
'FFCM1(-)',
'halogens',
'CHOF_G4',
'CHOCl_G4',
'CHOBr_G4',
'CHOFCl_G4',
'CHOFBr_G4',
'CHOFClBr_G4',
'DFT_QCI_thermo',
'2-BTP_G4',
'thermo_DFT_CCSDTF12_BAC',
'SulfurHaynes'
]

kin_families = ['Intra_R_Add_Endocyclic']


In [9]:
database = RMGDatabase()
database.load(
            path = settings['database.directory'],
            thermo_libraries = thermo_libs,
            transport_libraries = [],
            reaction_libraries = [],
            seed_mechanisms = [],#['BurkeH2O2inN2','ERC-FoundationFuelv0.9'],
            kinetics_families = kin_families,
            kinetics_depositories = ['training'],
            #frequenciesLibraries = self.statmechLibraries,
            depository = False, # Don't bother loading the depository information, as we don't use it
        )


In [21]:
database.kinetics.families

{'Intra_R_Add_Endocyclic': <ReactionFamily "Intra_R_Add_Endocyclic">}

In [22]:
# 'Intra_R_Add_Endocyclic' 
family_to_train = "Intra_R_Add_Endocyclic"
family = database.kinetics.families[family_to_train]

In [23]:
family.clean_tree()

In [24]:
start = time.time()
family.generate_tree(thermo_database=database.thermo,
                     nprocs=1,
                     new_fraction_threshold_to_reopt_node=0.25,
                     max_batch_size=800,
                     extension_iter_max=2,
                     extension_iter_item_cap=100)

end = time.time()
print(end-start)

ERROR:root:getting reaction matches
ERROR:root:building tree with 800 rxns
ERROR:root:800.0
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early
ERROR:root:Recursion item cap hit not splitting 3 reactions at iter 4 with 114 items
ERROR:root:Recursion item cap hit not splitting 2 reactions at iter 4 with 120 items
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved term

389.80308771133423


In [25]:
start = time.time()
family.check_tree()
end = time.time()
print(end-start)

0.6420941352844238


In [26]:
start = time.time()
family.regularize(thermo_database=database.thermo)
end = time.time()
print(end-start)

127.92794299125671


In [27]:
start = time.time()
templateRxnMap = family.get_reaction_matches(thermo_database=database.thermo,remove_degeneracy=True,
                                             get_reverse=True,exact_matches_only=False,fix_labels=True)
end = time.time()
print(end-start)

125.54991817474365


In [34]:
rxns = templateRxnMap['Root_N-1R-inRing_N-3R->O_Ext-3CS-R_N-Sp-4R!H=3CCSS_Ext-3CS-R_Ext-5R!H-R_N-Sp-6R!H=5R!H_3CS-inRing_N-Sp-2R=1R']
rxns = np.array(rxns)

In [31]:
recipe = database.kinetics.families['Intra_R_Add_Endocyclic'].forward_recipe.actions

In [36]:
for i, rxn in enumerate(rxns):
    print(ArrheniusBM().fit_to_reactions(rxns[list(set(range(len(rxns))) - {i})], recipe=recipe))

# dlnks = np.array([
#                 np.log(
#                     ArrheniusBM().fit_to_reactions(rxns[list(set(range(len(rxns))) - {i})], recipe=recipe)
#                     .to_arrhenius(rxn.get_enthalpy_of_reaction(Tref))
#                     .get_rate_coefficient(T=Tref) / rxn.get_rate_coefficient(T=Tref)
#                 ) for i, rxn in enumerate(rxns)

ArrheniusBM(A=(6.09e+08,'s^-1'), n=0.695, w0=(289.5,'kJ/mol'), E0=(90.5131,'kJ/mol'), Tmin=(300,'K'), Tmax=(2000,'K'), comment="""Fitted to 1 reactions at temperatures: [ 300.  500.  600.  700.  800.  900. 1000. 1100. 1200. 1500. 2000.]""")
ArrheniusBM(A=(2.992e+11,'s^-1'), n=0.67, w0=(289.5,'kJ/mol'), E0=(289.5,'kJ/mol'), Tmin=(300,'K'), Tmax=(2000,'K'), comment="""Fitted to 1 reactions at temperatures: [ 300.  500.  600.  700.  800.  900. 1000. 1100. 1200. 1500. 2000.]""")


/Users/nora/opt/anaconda3/envs/rmg_env/lib/python3.7/site-packages/scipy/optimize/minpack.py:834: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


In [5]:
BM_fit_2 = Reaction(kinetics = ArrheniusBM(A=(2.992e+11,'s^-1'), n=0.67, w0=(289.5,'kJ/mol'), E0=(289.5,'kJ/mol'), Tmin=(300,'K'), Tmax=(2000,'K'), comment="""Fitted to 1 reactions at temperatures: [ 300.  500.  600.  700.  800.  900. 1000. 1100. 1200. 1500. 2000.]"""))
BM_fit_1 = Reaction(kinetics = ArrheniusBM(A=(6.09e+08,'s^-1'), n=0.695, w0=(289.5,'kJ/mol'), E0=(90.5131,'kJ/mol'), Tmin=(300,'K'), Tmax=(2000,'K'), comment="""Fitted to 1 reactions at temperatures: [ 300.  500.  600.  700.  800.  900. 1000. 1100. 1200. 1500. 2000.]"""))


Tref=1000

print(BM_fit_2.kinetics.to_arrhenius(training_rxn_2.get_enthalpy_of_reaction(Tref)).get_rate_coefficient(Tref))  #problem, always becomes nan
print(BM_fit_1.kinetics.to_arrhenius(training_rxn_1.get_enthalpy_of_reaction(Tref)).get_rate_coefficient(Tref))

nan
12942146124.027784


In [14]:
#BM_fit_2.kinetics
BM_fit_2.kinetics.to_arrhenius(training_rxn_2.get_enthalpy_of_reaction(Tref)).get_rate_coefficient(1000)

nan

In [15]:
training_rxn_2.get_enthalpy_of_reaction(Tref)

-177996.41024

Problem: 

in family.py, make_bm_rules_from_template calls _make_rule(rr)

in _make_rule(rr), does a BM fitting with ArrheniusBM().fit_to_reactions().

ArrheniusBM().